In [1]:
# -*- coding: utf-8 -*-
"""
Carlos Angel

Examples to run main functions of the package.
v0.1 : 2023/09/15
v1.0 : 2023/11/05
v2.0 : 2023/12/12 (Added celltype and new reconstruct functions)
v3.0 : 2024/01/02 (Added synthetic data module)
Python version: 3.10.12
"""


import importlib
import numpy as np
import scipy as scp
import torch
import pandas as pd
import hicstraw
from pathlib import Path
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.pyplot as plt 
import seaborn as sns
import sys
from math import trunc
import os

sys.path.append('/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning')

from src import utils
from src import globals
from src import submatrices
from src import reconstruct
np.set_printoptions(threshold=sys.maxsize)

Importing pcSAC libraries...


Whole and non-split chains options:
The point here is that we wanted to split the amount of chains to start reconstructing one forward and one reverse.



In [4]:
#file and bool_res are default
#lrbin refers to the number of bins in one HR bin, so if HR is 5kb and LR is 20kb, lrbin=4
utils.run_genData(
    chr=globals.chromosome,
    start=globals.start,
    end=globals.end,
    downsample_factor=16,
    hic_res=5000, 
    lrbin=4,
    split_chains=False
)

Submitted batch job 38461777


In [6]:
utils.run_genData(
    chr=globals.chromosome,
    start=globals.start,
    end=globals.end,
    downsample_factor=1,
    hic_res=5000, 
    lrbin=4,
    split_chains=False
)

Submitted batch job 38616254


In [6]:
#Split chains is only available for whole matrices, do not use for submatrices
utils.run_pcSAC(downsampling_factor=16,
                         lrf=20,
                         run_option="whole",
                         number_chains=10000,
                         split_chains=False,
                         region_size=globals.size,
                         hic_res=globals.hic_res)

Submitted batch job 38462685


In [7]:
#Split chains is only available for whole matrices, do not use for submatrices
utils.run_pcSAC(downsampling_factor=1,
                         lrf=20,
                         run_option="whole",
                         number_chains=1000,
                         split_chains=False,
                         region_size=globals.size,
                         hic_res=globals.hic_res)

Submitted batch job 38617236


In [ ]:
#add comparison test
# it is also available on utils module, must be run with that module but sisnce we just implemented it we dont want to reload the kernel
reconstruct.run_reconstruction(downsampling_factor=16,
                                lrf=20,
                                run_option="whole",
                                number_chains=10000,
                                region_size = globals.size,
                                hic_res = globals.hic_res,
                                chromosome = globals.chromosome,
                                start=globals.start)

In [5]:
#splitting matrices is interfeiring because the amount of chains so we need to implement some changes
reconstruct.run_reconstruction(downsampling_factor=16,
                                lrf=20,
                                run_option="whole",
                                number_chains=10000,
                                region_size = globals.size,
                                hic_res = globals.hic_res,
                                chromosome = globals.chromosome,
                                start=globals.start)

Submitted batch job 38557231


In [3]:
#tmp for Gamze
reconstruct.run_reconstruction(downsampling_factor=16,
                                lrf=20,
                                run_option="whole",
                                number_chains=5000,
                                region_size = globals.size,
                                hic_res = globals.hic_res,
                                chromosome = globals.chromosome,
                                start=globals.start)

Submitted batch job 38513121


### Split chains option

In [8]:
#Split chains is only available for whole matrices, do not use for submatrices
utils.run_genData(
    chr=globals.chromosome,
    start=globals.start,
    end=globals.end,
    downsample_factor=16,
    hic_res=5000, 
    lrbin=4,
    split_chains=True,
    file=globals.hic_file
)

Submitted batch job 41200767


In [13]:
#Split chains is only available for whole matrices, do not use for submatrices
utils.run_pcSAC(downsampling_factor=16,
                         lrf=20,
                         run_option="whole",
                         number_chains=10000,
                         split_chains=True,
                         region_size=globals.size,
                         hic_res=globals.hic_res)


Submitted batch job 38477635


### Toy examples


In [3]:
utils.run_genData(
    chr=globals.chromosome,
    start=globals.start,
    end=globals.start + 600000,
    downsample_factor=16,
    hic_res=5000, 
    lrbin=4,
    split_chains=True
)

Submitted batch job 40206755


In [9]:
#Testing with all interactions in the specific range 
#Remember 1100 chains

utils.run_pcSAC(downsampling_factor=16,
                         lrf=20,
                         run_option="whole",
                         number_chains=1100,
                         split_chains=False,
                         region_size=globals.size,
                         hic_res=globals.hic_res)

Submitted batch job 40211847


### New compiler version 

## 10kb resolution workflow


### Tested regions  chr20

In [15]:
# Generate data for each region

my_regions = ['chr20:100000-2100000', 
              'chr20:20000000-21500000', 
              'chr20:42000000-43800000', 
              'chr20:2100000-3800000',
              'chr20:5000000-6950000',
              'chr20:43801000-45710000',
              'chr20:47622000-49500000',
              'chr20:45710000-47620000',
              ]

downsampling_factor = [16,25,50,100] 
n_chains = 10000
cell = 'GM12878'
hic_res = 10000

for region in my_regions:
    chrom, positions = region.split(':')
    chrom = chrom.replace('chr', '')
    start, end = map(int, positions.split('-'))
    print("Generating data for region:")
    print(f"Chromosome:{chrom}, {start}, {end}")
    for ds in downsampling_factor:
        print(f"Downsampling factor: {ds}")
        utils.run_genData(
        chr=globals.chromosome,
        start=start,
        end=end,
        downsample_factor=ds,
        hic_res=hic_res, 
        lrbin=2,
        split_chains=False,
        file=globals.hic_file
    )
    print("Input data generated\n")

Generating data for region:
Chromosome:20, 100000, 2100000
Downsampling factor: 16
Submitted batch job 41699740
Downsampling factor: 25
Submitted batch job 41699741
Downsampling factor: 50
Submitted batch job 41699742
Downsampling factor: 100
Submitted batch job 41699743
Input data generated

Generating data for region:
Chromosome:20, 20000000, 21500000
Downsampling factor: 16
Submitted batch job 41699744
Downsampling factor: 25
Submitted batch job 41699745
Downsampling factor: 50
Submitted batch job 41699746
Downsampling factor: 100
Submitted batch job 41699747
Input data generated

Generating data for region:
Chromosome:20, 42000000, 43800000
Downsampling factor: 16
Submitted batch job 41699748
Downsampling factor: 25
Submitted batch job 41699749
Downsampling factor: 50
Submitted batch job 41699750
Downsampling factor: 100
Submitted batch job 41699751
Input data generated

Generating data for region:
Chromosome:20, 2100000, 3800000
Downsampling factor: 16
Submitted batch job 41699752

In [16]:
#Generating ensambles for each region

#for 5k we only tested 16 and 25, if it works for tomorrow we can run the rest
for region in my_regions:
    chrom, positions = region.split(':')
    chrom = chrom.replace('chr', '')
    start, end = map(int, positions.split('-'))
    for ds in downsampling_factor:
        utils.run_pcSAC(
            downsampling_factor=ds,
            lrf=20,
            run_option="whole",
            number_chains=n_chains,
            split_chains=False,
            region_size=int(end-start),
            hic_res=hic_res,
            cell=cell)
        
        print("Running reconstruction for")
        print(f"Chromosome:{chrom}, {start}, {end}")
        print(f"Downsampling factor: {ds}\n")


Submitted batch job 41699887
Running reconstruction for
Chromosome:20, 100000, 2100000
Downsampling factor: 16

Submitted batch job 41699888
Running reconstruction for
Chromosome:20, 100000, 2100000
Downsampling factor: 25

Submitted batch job 41699889
Running reconstruction for
Chromosome:20, 20000000, 21500000
Downsampling factor: 16

Submitted batch job 41699890
Running reconstruction for
Chromosome:20, 20000000, 21500000
Downsampling factor: 25

Submitted batch job 41699891
Running reconstruction for
Chromosome:20, 42000000, 43800000
Downsampling factor: 16

Submitted batch job 41699892
Running reconstruction for
Chromosome:20, 42000000, 43800000
Downsampling factor: 25

Submitted batch job 41699893
Running reconstruction for
Chromosome:20, 2100000, 3800000
Downsampling factor: 16

Submitted batch job 41699894
Running reconstruction for
Chromosome:20, 2100000, 3800000
Downsampling factor: 25

Submitted batch job 41699895
Running reconstruction for
Chromosome:20, 5000000, 6950000
Do

In [3]:
#Reconstructing matrices from ensambles per region

for region in my_regions:
    for ds in downsampling_factor:
        chrom, positions = region.split(':')
        chrom = chrom.replace('chr', '')
        start, end = map(int, positions.split('-'))
        print("Generating metrics for region:")
        print(chrom, start, end)
        print(f"With downsampling factor:{ds}")


        reconstruct.run_reconstruction(downsampling_factor=ds,
                                    lrf=20,
                                    run_option="whole",
                                    number_chains=n_chains,
                                    region_size = int(end - start),
                                    hic_res=hic_res,
                                    chromosome=str(chrom),
                                    start=start,
                                    cell_t=cell)

Generating metrics for region:
20 100000 2100000
With downsampling factor:16
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '2000000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10005', '-c', '20', '-s', '100000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41647421
Generating metrics for region:
20 100000 2100000
With downsampling factor:25
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '25', '-z', '2000000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10005', '-c', '20', '-s', '100000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41647422
Generating metrics for region:
20 100000 2100000
With downsampling factor:50
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '50', '-z', '2000000', '-h', '

## Regions Different chromosomes

In [2]:
importlib.reload(utils)

<module 'src.utils' from '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/src/utils.py'>

In [3]:
chromosome_lengths_hg19 = {
    'chr1': 249250621,
    'chr2': 243199373,
    'chr3': 198022430,
    'chr4': 191154276,
    'chr5': 180915260,
    'chr6': 171115067,
    'chr7': 159138663,
    'chr8': 146364022,
    'chr9': 141213431,
    'chr10': 135534747,
    'chr11': 135006516,
    'chr12': 133851895,
    'chr13': 115169878,
    'chr14': 107349540,
    'chr15': 102531392,
    'chr16': 90354753,
    'chr17': 81195210,
    'chr18': 78077248,
    'chr19': 59128983,
    'chr20': 63025520,
    'chr21': 48129895,
    'chr22': 51304566,
    'chrX': 155270560,
    'chrY': 59373566
}


In [4]:
import random

def generate_regions(chr_num, num_regions, min_size=1_500_000, max_size=1_800_000):
    regions = []
    lengths_set = set()
    chr = "chr" + str(chr_num)
    chromosome_length = chromosome_lengths_hg19[chr]

    while len(regions) < num_regions:
        start = random.randint(1, chromosome_length - max_size)

        min_size_adjusted = (min_size + 49) // 50 * 50
        max_size_adjusted = max_size // 50 * 50

        num_intervals = (max_size_adjusted - min_size_adjusted) // 50 + 1

        random_interval = random.randint(0, num_intervals - 1)
        length = min_size_adjusted + random_interval * 50

        end = start + length

        if end > chromosome_length:
            end = chromosome_length

        #check if this length has already been used
        if length not in lengths_set:
            lengths_set.add(length)
            regions.append(f'{chr}:{start}-{end}')

    return regions


In [2]:
gen_reg = False # just once, we are saving the regions in a file so we can use them later
if gen_reg:
    for i in range(1,23):
        chromosome_selected = int(i)
        my_regions = generate_regions(chromosome_selected, 10)
        dir_cwd = os.path.join(os.getcwd(), "chr_regions")
        os.makedirs(dir_cwd, exist_ok=True)
        chromosome_regions = "regions_chr"+str(chromosome_selected)+".txt"
        chromosome_regions = os.path.join(dir_cwd, chromosome_regions)
        with open(chromosome_regions, 'w') as f:
            for item in my_regions:
                f.write("%s\n" % item)

        downsampling_factor = [16] # if we want to test more downsampling factors we can add them here
        n_chains = 10000
        cell = 'GM12878'
        hic_res = 10000

        for region in my_regions:
            chrom, positions = region.split(':')
            chrom = chrom.replace('chr', '')
            start, end = map(int, positions.split('-'))
            print("Generating data for region:")
            print(f"Chromosome:{chrom}, {start}, {end}")
            for ds in downsampling_factor:
                print(f"Downsampling factor: {ds}")
                utils.run_genData(
                chr=chromosome_selected,
                start=start,
                end=end,
                downsample_factor=ds,
                hic_res=hic_res, 
                lrbin=2,
                split_chains=False,
                file=globals.hic_file
            )
            print("Input data generated\n")

In [6]:
regions_dir="/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/src/chr_regions/"
regions_files = os.listdir(regions_dir)
regions_files.sort()

In [11]:

for i in regions_files:    

    my_regions = []
    with open(os.path.join(regions_dir,i), 'r') as file:
        for line in file:
            my_regions.append(line.strip())
            
    downsampling_factor = [16] # if we want to test more downsampling factors we can add them here
    for region in my_regions:
        chrom, positions = region.split(':')
        chrom = chrom.replace('chr', '')
        start, end = map(int, positions.split('-'))
        for ds in downsampling_factor:
            utils.run_pcSAC(
                chromosome=str("chr")+chrom,
                downsampling_factor=ds,
                lrf=20,
                run_option="whole",
                number_chains=n_chains,
                split_chains=False,
                region_size=int(end-start),
                hic_res=hic_res,
                cell=cell)
                
                print("Running reconstruction for")
                print(f"Chromosome:{chrom}, {start}, {end}")
                print(f"Downsampling factor: {ds}\n")




NameError: name 'n_chains' is not defined

In [7]:
for i in regions_files:    
    my_regions = []
    with open(os.path.join(regions_dir,i), 'r') as file:
        for line in file:
            my_regions.append(line.strip())
    downsampling_factor = [16] # if we want to test more downsampling factors we can add them here
    for region in my_regions:
        chrom, positions = region.split(':')
        chrom = chrom.replace('chr', '')
        start, end = map(int, positions.split('-'))
        print(chrom, start, end, int(end - start))
        reconstruct.run_reconstruction(downsampling_factor=int(16),
                                        lrf=20,
                                        run_option="whole",
                                        number_chains=10000,
                                        region_size = int(end - start),
                                        hic_res=10000,
                                        chromosome=chrom, #must be an int
                                        start=start)

regions_chr1.txt
regions_chr10.txt
10 51460966 53161766 1700800
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '1700800', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '10', '-s', '51460966', '-i', '160', '-t', 'GM12878']
Submitted batch job 41880001
10 14897755 16422355 1524600
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '1524600', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '10', '-s', '14897755', '-i', '160', '-t', 'GM12878']
Submitted batch job 41880002
10 81888886 83675936 1787050
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '1787050', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '10', '-s', '81888886', '-i', '160', '-t', 'GM128

## 10kb resolution workflow 4:1


In [10]:
importlib.reload(utils)

<module 'src.utils' from '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/src/utils.py'>

In [11]:
# Generate data for each region

my_regions = [ 
              'chr20:2100000-3800000',
              'chr20:47622000-49500000',
              ]

downsampling_factor = [16,25,50,100] 
n_chains = 10000
cell = 'GM12878'
hic_res = 10000

for region in my_regions:
    chrom, positions = region.split(':')
    chrom = chrom.replace('chr', '')
    start, end = map(int, positions.split('-'))
    print("Generating data for region:")
    print(f"Chromosome:{chrom}, {start}, {end}")
    for ds in downsampling_factor:
        print(f"Downsampling factor: {ds}")
        utils.run_genData(
        chr=globals.chromosome,
        start=start,
        end=end,
        downsample_factor=ds,
        hic_res=hic_res, 
        lrbin=4,
        split_chains=False,
        file=globals.hic_file
    )
    print("Input data generated\n")

Generating data for region:
Chromosome:20, 2100000, 3800000
Downsampling factor: 16
Submitted batch job 41698339
Downsampling factor: 25
Submitted batch job 41698340
Downsampling factor: 50
Submitted batch job 41698341
Downsampling factor: 100
Submitted batch job 41698342
Input data generated

Generating data for region:
Chromosome:20, 47622000, 49500000
Downsampling factor: 16
Submitted batch job 41698343
Downsampling factor: 25
Submitted batch job 41698344
Downsampling factor: 50
Submitted batch job 41698345
Downsampling factor: 100
Submitted batch job 41698346
Input data generated



In [12]:
for region in my_regions:
    chrom, positions = region.split(':')
    chrom = chrom.replace('chr', '')
    start, end = map(int, positions.split('-'))
    for ds in downsampling_factor:
        utils.run_pcSAC(
            downsampling_factor=ds,
            lrf=40,
            run_option="whole",
            number_chains=n_chains,
            split_chains=False,
            region_size=int(end-start),
            hic_res=hic_res,
            cell=cell)
        
        print("Running reconstruction for")
        print(f"Chromosome:{chrom}, {start}, {end}")
        print(f"Downsampling factor: {ds}\n")

Submitted batch job 41698349
Running reconstruction for
Chromosome:20, 2100000, 3800000
Downsampling factor: 16

Submitted batch job 41698350
Running reconstruction for
Chromosome:20, 2100000, 3800000
Downsampling factor: 25

Submitted batch job 41698351
Running reconstruction for
Chromosome:20, 2100000, 3800000
Downsampling factor: 50

Submitted batch job 41698352
Running reconstruction for
Chromosome:20, 2100000, 3800000
Downsampling factor: 100

Submitted batch job 41698353
Running reconstruction for
Chromosome:20, 47622000, 49500000
Downsampling factor: 16

Submitted batch job 41698354
Running reconstruction for
Chromosome:20, 47622000, 49500000
Downsampling factor: 25

Submitted batch job 41698355
Running reconstruction for
Chromosome:20, 47622000, 49500000
Downsampling factor: 50

Submitted batch job 41698356
Running reconstruction for
Chromosome:20, 47622000, 49500000
Downsampling factor: 100



In [13]:
my_regions

['chr20:2100000-3800000', 'chr20:47622000-49500000']

In [14]:
for region in my_regions:
    for ds in downsampling_factor:
        chrom, positions = region.split(':')
        chrom = chrom.replace('chr', '')
        start, end = map(int, positions.split('-'))
        print("Generating metrics for region:")
        print(chrom, start, end)
        print(f"With downsampling factor:{ds}")


        reconstruct.run_reconstruction(downsampling_factor=ds,
                                    lrf=40,
                                    run_option="whole",
                                    number_chains=n_chains,
                                    region_size = int(end - start),
                                    hic_res=hic_res,
                                    chromosome=str(chrom),
                                    start=start,
                                    cell_t=cell)

Generating metrics for region:
20 2100000 3800000
With downsampling factor:16
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '1700000', '-h', '10000', '-l', '40', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '2100000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41699317
Generating metrics for region:
20 2100000 3800000
With downsampling factor:25
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '25', '-z', '1700000', '-h', '10000', '-l', '40', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '2100000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41699318
Generating metrics for region:
20 2100000 3800000
With downsampling factor:50
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '50', '-z', '1700000', '-

### Synthetic data as gold standard 

This section is dedicated only to the analysis of synth data by different downsamples

In [2]:
sys.path.append('../synthetic/')
from synth_src import utils as synth_utils
from synth_src import globals as synth_globals
importlib.reload(synth_utils)
importlib.reload(synth_globals)


<module 'synth_src.globals' from '/gpfs/commons/groups/gursoy_lab/cangel/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/src/../synthetic/synth_src/globals.py'>

Generating gold standard synthetic matrix

In [82]:
#Generating synthetic data
synth_utils.generate_synthetic_chains(monomers = 200, cells = 10000, type="COMPLEX", dc=1500)


Running synthetic chain generation with the following parameters:
CMD:  sbatch /gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/synthetic/generate_chains.sh 200 10000 1500 COMPLEX chains_200-monomers_10000-cells_1500-dc_COMPLEX
Number of monomers:  200
Number of cells:  10000
Interaction distance:  1500
Type of chain:  COMPLEX
Output directory:  /gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/results/synthetic/chains_200-monomers_10000-cells_1500-dc_COMPLEX
Success in generating synthetic chains: b'Submitted batch job 41590376\n'


0

Downsampling generated chains to create LR matrices (input)

In [5]:
chains_downsample_factors = [100,250,1000,2500,3750,5000]
for ds in chains_downsample_factors:
    print(f"Downsampling: {ds} chains")

    synth_utils.downsample_synthetic_chains(all_chains=10000,
                                            chains_sample=ds)
    
    print(f"Generating 3D ensambles with downsampled chains: {ds}")
    
    synth_utils.pcSAC_synthetic(downsampled_chains=ds,
                                run_option="whole",
                                number_chains=10000,
                                split_chains=False)

Downsampling: 100 chains
Computing HR contact map 1
Computing HR contact map 2
Computing HR contact map 3
Computing HR contact map 4
Computing HR contact map 5
Computing HR contact map 6
Computing HR contact map 7
Computing HR contact map 8
Computing HR contact map 9
Computing HR contact map 10
Computing HR contact map 11
Computing HR contact map 12
Computing HR contact map 13
Computing HR contact map 14
Computing HR contact map 15
Computing HR contact map 16
Computing HR contact map 17
Computing HR contact map 18
Computing HR contact map 19
Computing HR contact map 20
Computing HR contact map 21
Computing HR contact map 22
Computing HR contact map 23
Computing HR contact map 24
Computing HR contact map 25
Computing HR contact map 26
Computing HR contact map 27
Computing HR contact map 28
Computing HR contact map 29
Computing HR contact map 30
Computing HR contact map 31
Computing HR contact map 32
Computing HR contact map 33
Computing HR contact map 34
Computing HR contact map 35
Comp

In [32]:
importlib.reload(synth_utils)
chains_downsample_factors = [100,250,1000,2500,3750,5000]
for ds in chains_downsample_factors:
    synth_utils.read_pcSAC_reconstruction(downsampled_chains=int(ds),distance=1500)


Submitted batch job 41645486
Submitted batch job 41645487
Submitted batch job 41645488
Submitted batch job 41645489
Submitted batch job 41645490
Submitted batch job 41645491


### Detailed regions 

### chr16: 57000000 - 59500000

In [7]:
dSample = [16,25,50,100]
lr = 20
resolution= 10000
start=57000000
end=59500000
for ds in dSample:
    utils.run_genData(
        chr=16,
        start=start,
        end=end,
        downsample_factor=ds,
        hic_res=resolution, 
        lrbin=2,
        split_chains=False
    )

Submitted batch job 40632397
Submitted batch job 40632398
Submitted batch job 40632399
Submitted batch job 40632400


In [3]:
#for 10K only 16
n_chains = [10000]
#dSample = [16,25,50,100]
dSample = [16]
lr = 20
resolution=10000
start=57000000
end=59500000
for n_c in n_chains:
    for ds in dSample:
        utils.run_pcSAC(downsampling_factor=ds,
                                lrf=lr,
                                run_option="whole",
                                number_chains=n_c,
                                split_chains=False,
                                region_size=int(end - start),
                                hic_res=resolution)

Submitted batch job 40636917


In [2]:
dSample = [16,25,50,100]
lr = 20
resolution= 10000
start=57000000
end=59500000

for i in dSample:
    reconstruct.run_reconstruction(downsampling_factor=int(i),
                                    lrf=20,
                                    run_option="whole",
                                    number_chains=10000,
                                    region_size = int(end - start),
                                    hic_res=10000,
                                    chromosome = 16,
                                    start=start)

['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '2500000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '16', '-s', '57000000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316887
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '25', '-z', '2500000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '16', '-s', '57000000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316888
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '50', '-z', '2500000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '16', '-s', '57000000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316889
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/

### chr17:73500000-75800000

In [11]:
dSample = [16,25,50,100]
lr = 20
resolution= 10000
start=73500000
end=75800000
chrn=17
for ds in dSample:
    utils.run_genData(
        chr=chrn,
        start=start,
        end=end,
        downsample_factor=ds,
        hic_res=resolution, 
        lrbin=2,
        split_chains=False
    )

Submitted batch job 40632483
Submitted batch job 40632484
Submitted batch job 40632485
Submitted batch job 40632486


In [4]:
n_chains = [10000]
dSample = [16]
lr = 20
resolution= 10000
start=73500000
end=75800000
chrn=17
for n_c in n_chains:
    for ds in dSample:
        utils.run_pcSAC(downsampling_factor=ds,
                                lrf=lr,
                                run_option="whole",
                                number_chains=n_c,
                                split_chains=False,
                                region_size=int(end - start),
                                hic_res=resolution)

Submitted batch job 40636921


In [3]:
dSample = [16,25,50,100]
lr = 20
resolution= 10000
start=73500000
end=75800000
chrn=17

for i in dSample:
    reconstruct.run_reconstruction(downsampling_factor=int(i),
                                    lrf=20,
                                    run_option="whole",
                                    number_chains=1000,
                                    region_size = int(end - start),
                                    hic_res=10000,
                                    chromosome = chrn,
                                    start=start)

['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '2300000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '1000', '-c', '17', '-s', '73500000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316893
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '25', '-z', '2300000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '1000', '-c', '17', '-s', '73500000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316894
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '50', '-z', '2300000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '1000', '-c', '17', '-s', '73500000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316895
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/Par

### chr22:23000000-24900000

In [14]:
dSample = [16,25,50,100]
lr = 20
resolution= 10000
start=23000000
end=24900000
chrn=22
for ds in dSample:
    utils.run_genData(
        chr=chrn,
        start=start,
        end=end,
        downsample_factor=ds,
        hic_res=resolution, 
        lrbin=2,
        split_chains=False
    )

Submitted batch job 40632505
Submitted batch job 40632506
Submitted batch job 40632507
Submitted batch job 40632508


In [5]:
n_chains = [10000]
dSample = [16]
lr = 20
resolution= 10000
start=23000000
end=24900000
chrn=22
for n_c in n_chains:
    for ds in dSample:
        utils.run_pcSAC(downsampling_factor=ds,
                                lrf=lr,
                                run_option="whole",
                                number_chains=n_c,
                                split_chains=False,
                                region_size=int(end - start),
                                hic_res=resolution)

Submitted batch job 40636925


In [4]:
dSample = [16,25,50,100]
lr = 20
resolution= 10000
start=23000000
end=24900000
chrn=22

for i in dSample:
    reconstruct.run_reconstruction(downsampling_factor=int(i),
                                    lrf=20,
                                    run_option="whole",
                                    number_chains=1000,
                                    region_size = int(end - start),
                                    hic_res=resolution,
                                    chromosome = chrn,
                                    start=start)

['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '1900000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '1000', '-c', '22', '-s', '23000000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316902
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '25', '-z', '1900000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '1000', '-c', '22', '-s', '23000000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316903
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '50', '-z', '1900000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '1000', '-c', '22', '-s', '23000000', '-i', '160', '-t', 'GM12878']
Submitted batch job 41316904
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/Par

## Different cell lines

11/10/23

v2.0 added cell_type in ensambles_3D and modified part 2 of the general pipeline to identify cell type

In [2]:
import utils

In [3]:
dSample = [3,5,10,20] # proportional to original ones
for ds in dSample:
    utils.run_genData(
        file=globals.hic_K562,
        chr=globals.chromosome,
        start=globals.start,
        end=globals.end,
        downsample_factor=ds,
        hic_res=10000, 
        lrbin=2,
        split_chains=False
    )



Submitted batch job 40856708
Submitted batch job 40856709
Submitted batch job 40856710
Submitted batch job 40856711


In [4]:
for ds in dSample:    
    utils.run_genData(
        file=globals.hic_IMR90,
        chr=globals.chromosome,
        start=globals.start,
        end=globals.end,
        downsample_factor=ds,
        hic_res=10000, 
        lrbin=2,
        split_chains=False
    )

Submitted batch job 40856718
Submitted batch job 40856719
Submitted batch job 40856720
Submitted batch job 40856721


In [5]:
for ds in dSample:    
    utils.run_pcSAC(downsampling_factor=ds,
                            lrf=20,
                            run_option="whole",
                            number_chains=10000,
                            split_chains=False,
                            region_size=globals.size,
                            hic_res=10000,
                            cell="K562")

Submitted batch job 40857017
Submitted batch job 40857018
Submitted batch job 40857019
Submitted batch job 40857020


In [6]:
for ds in dSample:    
    utils.run_pcSAC(downsampling_factor=ds,
                            lrf=20,
                            run_option="whole",
                            number_chains=10000,
                            split_chains=False,
                            region_size=globals.size,
                            hic_res=10000,
                            cell="IMR90")

Submitted batch job 40857024
Submitted batch job 40857025
Submitted batch job 40857026
Submitted batch job 40857027


In [2]:
dSample = [3,5,10,20] 
for i in dSample:
    reconstruct.run_reconstruction(downsampling_factor=int(i),
                                    lrf=20,
                                    run_option="whole",
                                    number_chains=10000,
                                    region_size = globals.size,
                                    hic_res=10000,
                                    chromosome = globals.chromosome,
                                    start=globals.start,
                                    cell_t="K562")


['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '3', '-z', '2000000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '100000', '-i', '170', '-t', 'K562']
Submitted batch job 40883178
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '5', '-z', '2000000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '100000', '-i', '170', '-t', 'K562']
Submitted batch job 40883179
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '10', '-z', '2000000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '100000', '-i', '170', '-t', 'K562']
Submitted batch job 40883180
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/0

In [3]:
dSample = [3,5,10,20] 
for i in dSample:
    reconstruct.run_reconstruction(downsampling_factor=int(i),
                                    lrf=20,
                                    run_option="whole",
                                    number_chains=10000,
                                    region_size = globals.size,
                                    hic_res=10000,
                                    chromosome = globals.chromosome,
                                    start=globals.start,
                                    cell_t="IMR90")


['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '3', '-z', '2000000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '100000', '-i', '170', '-t', 'IMR90']
Submitted batch job 40883188
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '5', '-z', '2000000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '100000', '-i', '170', '-t', 'IMR90']
Submitted batch job 40883189
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '10', '-z', '2000000', '-h', '10000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '100000', '-i', '170', '-t', 'IMR90']
Submitted batch job 40883190
['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTunin

5kb


In [3]:
# Source utils not general utils 
utils.run_genData(
    chr=globals.chromosome,
    start=globals.start,
    end=globals.end,
    downsample_factor=16,
    hic_res=5000, 
    lrbin=4,
    split_chains=False,
    file=globals.hic_file
)

Submitted batch job 43693214


In [5]:
for ds in [25,50,100]:
    utils.run_genData(
        chr=globals.chromosome,
        start=globals.start,
        end=globals.end,
        downsample_factor=ds,
        hic_res=5000, 
        lrbin=4,
        split_chains=False,
        file=globals.hic_file
    )

Submitted batch job 43693233
Submitted batch job 43693234
Submitted batch job 43693235


In [7]:
for ds in [25,50,100]:
    utils.run_pcSAC(
                    chromosome=str("chr")+"20",
                    downsampling_factor=ds,
                    lrf=20,
                    run_option="whole",
                    number_chains=10000,
                    split_chains=False,
                    region_size=int(globals.end-globals.start),
                    hic_res=5000,
                    cell="GM12878")

Submitted batch job 43693239
Submitted batch job 43693240
Submitted batch job 43693241


In [3]:
for ds in [16,25,50,100]:
    reconstruct.run_reconstruction(downsampling_factor=int(ds),
                                    lrf=20,
                                    run_option="whole",
                                    number_chains=10000,
                                    region_size = globals.size,
                                    hic_res=5000,
                                    chromosome = globals.chromosome,
                                    start=globals.start,
                                    cell_t="GM12878")


['sbatch', '/gpfs/commons/home/cangel/g2lab/projects/01_12_23_highResolutionHiC/scripts/ParameterTuning/03.compare_matrices.sh', '-d', '16', '-z', '2000000', '-h', '5000', '-l', '20', '-o', 'whole', '-n', '10000', '-c', '20', '-s', '100000', '-i', '99', '-t', 'GM12878']
Submitted batch job 45595831
